 # **Exploração e análise de dados de crédito com SQL**

# Dados

Analisando o que cada coluna representa:

* idade = idade do cliente
* sexo = sexo do cliente(F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do cliente
* estado_civil = estado civil do cliente
* salario_anual = faixa salarial anual do cliente
* tipo_cartao = tipo de cartao do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* interacoes_12m = número de operações feitas pelo cliente nos últimos 12 meses
* meses_inativo_12m = quantidade de meses a qual o cliente não fez nenhuma operação
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor total das transações nos últimos 12 meses
* qtd_transacoes_12m = quantidade de transações feitas nos últimos 12 meses

A tabela foi criada no **AWS Athena** junto com o **S3 Bucket** com uma versão dos dados disponibilizados em: https://github.com/Pinho-11/Projeto_SQL/blob/main/dataset/creditos.xls

# Exploração de dados

Nessa etapa, farei uma breve análise do nosso conjunto de dados, visando entender como está distribuido os dados na tabela.

> Quantas linhas possui a base de dados?

```sql
SELECT COUNT(*) FROM credito
```

* Reposta: 2564 linhas

> Como é a base de dados?

```sql
SELECT * FROM credito LIMIT 10
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/select_all..PNG?raw=true)

* Nessas 10 primeiras linhas do dataset, é possivel perceber a presença de informações nulas na tabela(na), em particular nas colunas escolaridade e estado_civil.

> Qual o tipo de cada dado?

```sql
DESCRIBE credito
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/describe_credito.png?raw=true)

> Quais são os dados da coluna escolaridade?

```sql
SELECT DISTINCT escolaridade FROM credito
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/select_distinct_escolaridade.PNG?raw=true)

* Nessa coluna, além de apresentar os níveis de escolaridade, também pode-se notar a presença de dados nulos(na).

> Quais são os dados da coluna estado_civil?

```sql
SELECT DISTINCT estado_civil FROM credito
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/select_distinct_estado_civil.PNG?raw=true)

* Assim como na coluna *escolaridade*, na coluna *estado_civil* também está presente dados nulos.

> Quais são os dados da coluna salario_anual?

```sql
SELECT DISTINCT salario_anual FROM credito
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/select_distinct_salario_anual.PNG?raw=true)

* Nesse dataset, os salários anuais são apresentados por meio de uma faixa de renda. Essa coluna também apresenta dados nulos.

> Quantas pessoas possuem cada tipo de cartão?

```sql
SELECT COUNT(*) AS qtd_pessoas , tipo_cartao FROM credito
GROUP BY tipo_cartao
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/select_count_tipo_cartao.PNG?raw=true)

* Observa-se que não será necessário tratar valores nulos, além disso, vemos por meio dessa imagem a disparidade entre clientes com o cartão do tipo **blue** para os demais tipos.

# Análise de dados

> Qual a quantidade de homens e mulheres nessa base de dados?

```sql
SELECT COUNT(*) AS qtd_pessoas, sexo FROM credito GROUP BY sexo
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/count_sexo.PNG?raw=true)

* Por meio dessa querry, é possivel observar a disparidade entre clientes do sexo masculino e feminino. Há 562 homens a mais do que mulheres, eles representam cerca de 61% dos clientes, enquantos as mulheres representam 39%.

> Qual a relação entre o limite de crédito e a escolaridade?

```sql
SELECT AVG(limite_credito) AS media_limite, escolaridade FROM credito
WHERE escolaridade != 'na'
GROUP BY escolaridade
ORDER BY media_limite DESC
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/limite_escolaridade.PNG?raw=true)

* É possivel perceber que não existe uma relação direta entre o limite de crédito médio e a escolaridade do cliente. Nessa query, o limite médio de um cliente com curso de graduação é maior do que a de um com mestrado.

> O que podemos notar entre o limite de crédito e o salário anual?

```sql
SELECT AVG(limite_credito) AS media_limite , salario_anual FROM credito
WHERE salario_anual != 'na'
GROUP BY salario_anual
ORDER BY media_limite DESC
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/limite_salario_anual.PNG?raw=true)

* Podemos perceber que a limite de crédito médio e o salario anual são diretamente proporcionais, aqueles que ganham mais tem um limite de crédito maior do que aqueles que possuem uma renda inferior.

> Em qual categoria de renda anual podemos observar a maior oportunidade do aumento de transações?

```sql
SELECT COUNT(*) AS qtd_clientes, salario_anual, AVG(valor_transacoes_12m) AS media_transacoes FROM credito
WHERE tipo_cartao = 'blue' AND salario_anual != 'na'
GROUP BY salario_anual
ORDER BY qtd_clientes DESC
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/renda_vs_media_transacoes.PNG?raw=true)

* Podemos notar que por mais que 212 clientes possuam o cartão tipo blue e tenham uma renda anual superior a $120K, são esses os que obtiveram em média, o menor valor em transações nos últimos 12 meses. Por isso, devemos direcionar uma campanha de marketing especificamente para esse público.

> Visto que a maior parte dos clientes possui o cartão do tipo **blue**, qual seria a idade deles?

```sql
SELECT AVG(idade) AS idade_media, MIN(idade) as idade_minima, MAX(idade) AS idade_maxima FROM credito
WHERE tipo_cartao = 'blue'
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/idade_tipo_cartao_blue.PNG?raw=true)

* Como o cartão do tipo **blue** é o mais básico, era de se esperar uma disparidade grande entre a maior e a menor idade, nessa base de dados, essa disparidade é de 47 anos.

> Qual é o sexo que possui a maior razão entre o valor medio das transações e o limite do cartão?

```sql
SELECT AVG(valor_transacoes_12m) AS media_transacoes, AVG(limite_credito) AS limite_cartao, sexo FROM credito
GROUP BY sexo
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/media_transacoes_limite_sexo.PNG?raw=true)

* A razão entre o valor medio das transações e o limite do cartão para o sexo masculino é de 0,16(1.807,97/11.201,36), ou seja, os homens usam cerca de 16% do seu limite de crédito, enquanto as mulheres utilizam 32%(1.839,62/5.749). Ou seja, as mulheres gastam em média o dobro do que os homens quando esse valor é comparado ao limite do cartao.

> O estado civil do cliente interfere no quanto ele gasta?

```sql
SELECT  AVG(valor_transacoes_12m) AS media_transacoes , estado_civil FROM credito
WHERE estado_civil != 'na'
GROUP BY estado_civil
ORDER BY media_transacoes DESC
```

![](https://github.com/Pinho-11/Projeto_SQL/blob/main/queries_images/media_transacoes_estado_civil.PNG?raw=true)

* Por incrível que pareça, podemos perceber que quanto menor compromisso amoroso do cliente com um parceiro ou parceira, maior é o seu gasto.

# insights

* a maior parte dos clientes são do sexo masculino
* a escolaridade não tem um impacto direto no limite de crédito
* o salario anual é diretamente proporcional ao limite de crédito
* clientes com salario anual de $120K+ foram os que menos gastaram nos ultimos 12 meses
* o tipo cartão blue é o mais básico e abrange clientes dos 26 aos 73 anos
* na relação entre o valor medio das transações e o limite do cartão, as mulheres gastam o dobro do que os homens
* o estado civil interfere diretamente em quanto o cliente gasta

